<a href="https://colab.research.google.com/github/mojgan65/Fake-news-detection/blob/main/ClassificationFakeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.19.5
!pip install tensorflow==1.14.0
!pip install torch==1.9.0
!pip install scikit-learn==0.21.3
!pip install pytorch_pretrained_bert==0.6.2 pytorch-nlp==0.5.0     
!pip install tqdm==4.41.1
!pip install pandas==1.1.5
!pip install matplotlib==3.2.2
!pip install beautifulsoup4==4.6.3


#importing a few necessary packages and setting the DATA directory
DATA_DIR="."
import warnings
warnings.filterwarnings("ignore") 
import os
import numpy as np
import pickle
import tensorflow as tf





     |████████████████████████████████| 109.3 MB 50 kB/s 
     |████████████████████████████████| 3.1 MB 49.5 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 488 kB 72.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you

In [ ]:
# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)

In [5]:
# uploading and reading the dataset
# source for dataset: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
try :
    
    from google.colab import files
    # After downloading the dataset, upload the dataset here.
    uploaded = files.upload()
    df = pd.read_csv("en_dup.csv",engine='python', error_bad_lines=False)

except ModuleNotFoundError :
    
    # After downnloading the dataset, put the IMDB Dataset.csv file in Data folder.
    df = pd.read_csv("Data/en_dup.csv",engine='python', error_bad_lines=False)

Saving en_dup.csv to en_dup (1).csv


In [6]:
df.head()
df.value_counts('label')

label
F    3681
T    1878
U    1620
dtype: int64

In [7]:
from bs4 import BeautifulSoup
import re

def strip(text):
    soup = BeautifulSoup(text, "html.parser")
    text = re.sub('\[[^]]*\]', '', soup.get_text())
    pattern=r"[^a-zA-z0-9\s,']"
    text=re.sub(pattern,'',text)
    return text

df['content']=df['content'].apply(strip)

In [8]:
df.head()

,label,content,source,author,time
0,F,Coronavirus was created in a government lab as...,NaN,NaN,NaN
1,F,The lie that coronavirus came from a bat or a ...,NaN,NaN,NaN
2,F,The health experts had predicted the virus cou...,NaN,NaN,NaN
3,F,A video clip supposedly showed that the expone...,NaN,NaN,NaN
4,T,Almost 200 people in Italy died from the coron...,NaN,NaN,NaN


In [9]:
sentences = df['content']
sentence = ["[CLS] "+i+" [SEP]" for i in sentences]
sentence[0]

'[CLS] Coronavirus was created in a government lab as a bioweapon and then released on the people of China [SEP]'

In [10]:
# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Restricting the max size of Tokens to 512(BERT doest accept any more than this)
tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:510] , sentence))
print ("Tokenize the first sentence:")
print (tokenized_texts[0])
print(sentence[0])

100%|██████████| 231508/231508 [00:00<00:00, 940744.59B/s]


Tokenize the first sentence:
['[CLS]', 'corona', '##virus', 'was', 'created', 'in', 'a', 'government', 'lab', 'as', 'a', 'bio', '##we', '##ap', '##on', 'and', 'then', 'released', 'on', 'the', 'people', 'of', 'china', '[SEP]']
[CLS] Coronavirus was created in a government lab as a bioweapon and then released on the people of China [SEP]


In [11]:
from sklearn.preprocessing import LabelEncoder

#sentiment is positive and negative we need to convert it to 0,1
le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])


In [12]:
labels = list(df['label'])#storing the labels

In [13]:
# Set the maximum sequence length. 
MAX_LEN = 128

# Pad our input tokens so that everything has a uniform length
input_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, tokenized_texts)),
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [14]:
input_ids[0]

array([  101, 21887, 23350,  2001,  2580,  1999,  1037,  2231,  6845,
        2004,  1037, 16012,  8545,  9331,  2239,  1998,  2059,  2207,
        2006,  1996,  2111,  1997,  2859,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [15]:
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [17]:
# Select a batch size for training. 
batch_size = 16

# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)
                                             
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)